# XGBoost is one of the most popular machine learning algorithm these days. Regardless of the type of prediction task at hand; regression or classification.
## I will predict the students that have certified of > 50% Course by applying XGBoost algorithm!

# In this noote book I will explain the following:
## 1- Data visualization
## 2- Data prepration and dummy pandas function
## 3- XGBoost and its hyperprameters
## 4- XGBoost tree visualization
## 5- XGBoost features importance visualization

## Importing libraries & Data:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df_MIT= pd.read_csv('../input/course-study/appendix.csv')
df_MIT.head()

In [ ]:
df_MIT.info()

In [ ]:
df_MIT=df_MIT.drop(['% Certified','Course Number','Course Title','% Grade Higher Than Zero'],axis=1)
df_MIT

In [ ]:
df_MIT['Course Subject'].value_counts() #DUMMIES

In [ ]:
df_MIT['Instructors'].value_counts() #DELETE

### View nulls values in data set by heatmap:

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_MIT.isnull(),cmap="YlGnBu")

## A- Let's do Quick Data visualization!

In [ ]:
figure= plt.figure(figsize=(10,10))
sns.heatmap(df_MIT.corr(), annot=True,cmap="YlGnBu")
#To show the correlation between variables

In [ ]:
figure= plt.figure(figsize=(20,10))
sns.boxenplot(x='Course Subject',y='% Certified of > 50% Course Content Accessed',data=df_MIT,palette="Blues")

In [ ]:
figure= plt.figure(figsize=(20,10))
sns.boxenplot('Participants (Course Content Accessed)','Institution',data=df_MIT)

In [ ]:
df_pairplot_cols=df_MIT[['Institution','Audited (> 50% Course Content Accessed)','Year','% Certified of > 50% Course Content Accessed','Total Course Hours (Thousands)','% Female','% Male','Median Age']]
plt.figure(figsize=(20,20))
sns.pairplot(df_pairplot_cols,hue='Institution',palette='rainbow')

In [ ]:
sns.lmplot(x='Participants (Course Content Accessed)',y='Audited (> 50% Course Content Accessed)',data=df_MIT,col='Course Subject',hue='Institution',palette='coolwarm',
          aspect=0.6,size=8)

In [ ]:
sns.lmplot(x='Median Age',y='% Female',data=df_MIT,col='Course Subject',hue='Institution',palette='coolwarm',
          aspect=0.6,size=8)

In [ ]:
sns.lmplot(x='Median Age',y='% Male',data=df_MIT,col='Course Subject',hue='Institution',palette='coolwarm',
          aspect=0.6,size=8)

In [ ]:
x= df_MIT['Median Hours for Certification']
y= df_MIT['% Certified of > 50% Course Content Accessed']
cmap = sns.cubehelix_palette(light=1, as_cmap=True)
plt.figure(figsize=(10,10))
sns.kdeplot(x, y, cmap=cmap, shade=True);

In [ ]:
x= df_MIT['Median Age']
y= df_MIT['% Certified of > 50% Course Content Accessed']
plt.figure(figsize=(10,10))
sns.kdeplot(x, y, shade=True);

# B- Perperation of data:
## I will drop : 'Total Course Hours (Thousands)' , 'Certified' , 'Audited (> 50% Course Content Accessed)' , 'Instructors' , 'Launch Date' , '% Played Video'

In [ ]:
df_MIT.info()

In [ ]:
df_XGB = df_MIT.drop(['Total Course Hours (Thousands)','Certified','Audited (> 50% Course Content Accessed)','Instructors','Launch Date','% Played Video'],axis=1)

In [ ]:
df_XGB.info()

## Pandas get_dummies() :
### Pandas has a function which can turn a categorical variable into a series of zeros and ones, which makes them a lot easier to quantify and compare.

In [ ]:
Institution = pd.get_dummies(df_XGB['Institution'],drop_first=True)
CourseSubject = pd.get_dummies(df_XGB['Course Subject'],drop_first=True)
df_XGB.drop(['Institution','Course Subject'],axis=1,inplace=True)
df_XGB = pd.concat([df_XGB,Institution,CourseSubject],axis=1)


In [ ]:
df_XGB

## Check out if any nulls:

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_XGB.isnull(),cmap="YlGnBu")

# C- XGBOOST Predictions


## 1- Split data into train/test

In [ ]:
from sklearn.model_selection import train_test_split
x= df_XGB
y=df_XGB['% Certified of > 50% Course Content Accessed']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.4, random_state=109)

In [ ]:
import xgboost as xgb
train= xgb.DMatrix(x_train,label=y_train)
test = xgb.DMatrix(x_test, label= y_test)

### 2- XGBoost's hyperparameters
#### At this point, before building the model, you should be aware of the tuning parameters that XGBoost provides. Well, there are a plethora of tuning parameters for tree-based learners in XGBoost and you can read all about them here. But the most common ones that you should know are:

#### learning_rate: step size shrinkage used to prevent overfitting. Range is [0,1]
#### max_depth: determines how deeply each tree is allowed to grow during any boosting round.
#### subsample: percentage of samples used per tree. Low value can lead to underfitting.
#### colsample_bytree: percentage of features used per tree. High value can lead to overfitting.
#### n_estimators: number of trees you want to build.
#### objective: determines the loss function to be used like reg:linear for regression problems, reg:logistic for classification problems with only decision, binary:logistic for classification problems with probability.
#### XGBoost also supports regularization parameters to penalize models as they become more complex and reduce them to simple (parsimonious) models.

#### gamma: controls whether a given node will split based on the expected reduction in loss after the split. A higher value leads to fewer splits. Supported only for tree-based learners.
#### alpha: L1 regularization on leaf weights. A large value leads to more regularization.
#### lambda: L2 regularization on leaf weights and is smoother than L1 regularization.
#### It's also worth mentioning that though you are using trees as your base learners, you can also use XGBoost's relatively less popular linear base learners and one other tree learner known as dart. All you have to do is set the booster parameter to either gbtree (default),gblinear or dart.


In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.5, learning_rate = 0.2,
                max_depth = 7, alpha = 10, n_estimators = 75)

In [ ]:
xg_reg.fit(x_train,y_train)
preds = xg_reg.predict(x_test)


## 3- Take the error 

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

### That's mean 5.09 per 100 error!

## ______________________________________________________________________________________

## 4- Visualize Boosting Trees
### you can also visualize individual trees from the fully boosted model that XGBoost creates using the entire housing dataset. XGBoost has a plot_tree() function that makes this type of visualization easy. Once you train a model using the XGBoost learning API, you can pass it to the plot_tree() function along with the number of trees you want to plot using the num_trees argument.

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [20, 15]
plt.show()

# 5- XGBoost can count the number of times each feature is split or Feature importance features ordered according to how many times they appear. 

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [15,15]
plt.show()

#### Sourse : [XGBoost](https://www.datacamp.com/community/tutorials/xgboost-in-python)